# Лабораторна робота №4: Моделі класифікації та їх оцінка

### Мета роботи

Ознайомитися з основними моделями класифікації, а також навчитися оцінювати якість класифікаторів за різноманітними метриками.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Завантаження даних

Завантажте набори даних "Airline Passenger Satisfaction" для навчання (`train.csv`) та тестування (`test.csv`)  з [Kaggle](https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction) або з Google Classroom.

### 2. Обробка набору даних

1. З набору даних видаліть наступні стовпці:
    * `"Unnamed: 0"`
    * `"id"`
2. Заповніть всі пусті значення у наборі даних значенням `0`.
3. Перетворіть стовпці рядкових значень на стовпці, що мають числові значення.
4. Нормалізуйте значення кожної ознаки моделі з набору даних для навчання.

_У наборі даних кожен зі стовпчиків є ознакою моделі, окрім останнього під назвою `"satisfaction"`, який є цільовою змінною._

### 3. Навчання моделей

Натренуйте кожну з моделей:

* Наївний баєсів класифікатор [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
* Дерево рішень [Decision Tree](https://scikit-learn.org/stable/modules/tree.html)
* Логістична регресія [Logistic Regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)
* K-найближчі сусідів [K-Nearest Neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
* Випадковий ліс [Random Forest](https://scikit-learn.org/stable/modules/ensemble.html#random-forests-and-other-randomized-tree-ensembles)

_Для кожної з моделей також підрахуйте час виконання у мілісекундах, це буде необхідно для виконання 4-го завдання._

### 4. Підрахунок основних метрик

Для кожної з моделей підрахуйте наступні метрики:

* [Accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
* [Precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html)
* [Recall](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html)
* [F1-Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)
* Швидкість виконання (у мс.)

Отримані дані запишіть у `DataFrame` та візуалізуйте його у вигляді таблиці. Також можете додати до `DataFrame` `.style.background_gradient(cmap='Greens')`, що дозволить краще візуально порівнювати значення у кожному зі стовпчиків.

### 5. Візуалізація матриць невідповідностей

Для кожної з моделей візуалізуйте [матрицю невідповідності](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).